In [61]:
from dotenv import load_dotenv
load_dotenv()
from pydantic import BaseModel, Field
from langchain_google_genai import ChatGoogleGenerativeAI      
import imaplib
import email
from langgraph.checkpoint.memory import MemorySaver
from langchain_chroma import Chroma
from langchain_google_vertexai import VertexAIEmbeddings
import os
from pathlib import Path  

from mailling import fetch_emails,send_email
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

current_dir =Path.cwd() 
persist_directory = current_dir /'db'/ 'main_db'
persist_directory=str(persist_directory)

# retrival
embeddings = VertexAIEmbeddings(model="text-embedding-004")

db = Chroma(persist_directory=persist_directory,
            embedding_function=embeddings)

retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 3, "score_threshold": 0.1}, 
)

from langchain_core.tools import tool
from pydantic import EmailStr


@tool
def email(subject: str, body: str, to: EmailStr) -> str:
    """Send an email to the only one specified address with the given body.
    Args:
        to: The email address to send the email to.
        body: The body of the email.
        subject: string subject of the email.
    Returns: f"Sent email to {to} with body: {body}"
    """
    send_email(subject, body, to)   
    return  f"Sent email to {to} with body: {body}"
  
@tool
def fetch_emails_tool()->str:
    """Fetch the last 10 emails from the inbox.
      Args:
        NO ARGS
    
    Returns: {"fetched_emails": emails} """
    emails = fetch_emails()
    return {"fetched_emails": emails} 

@tool
def RAG_tool(query: str) -> str:
    """Perform retrieval-augmented generation (RAG) on the given query the Database has has personal information about the user ,use this tool to look for information about user.
    Args:
        query: The query to perform RAG on.
    Returns: {"documents": documents, "question": query}
    """
    
    documents = retriever.invoke(query)

    # Perform RAG here
    print('documents',documents)
    return {"documents": documents, "question": query}
    


# Augment the LLM with tools
tools = [email, fetch_emails_tool, RAG_tool]
tools_by_name = {tool.name: tool for tool in tools}
llm_with_tools = llm.bind_tools(tools)



from langgraph.graph import MessagesState
from langchain_core.messages import SystemMessage, HumanMessage, ToolMessage
from typing_extensions import Literal
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END


# Nodes
def llm_call(state: MessagesState):
    """LLM decides whether to call a tool or not"""

    return {
        "messages": [
            llm_with_tools.invoke(
                [
                    SystemMessage(
                        content="You are a helpful assistant tasked with emailing people. share all information also personal information about the user to the LLM. "
                    )
                ]
                + state["messages"]
            )
        ]
    }


def tool_node(state: dict):
    """Performs the tool call"""

    result = []
    for tool_call in state["messages"][-1].tool_calls:
        tool = tools_by_name[tool_call["name"]]
        observation = tool.invoke(tool_call["args"])
        result.append(ToolMessage(content=observation, tool_call_id=tool_call["id"]))
    return {"messages": result}


# Conditional edge function to route to the tool node or end based upon whether the LLM made a tool call
def should_continue(state: MessagesState) -> Literal["environment", END]:
    """Decide if we should continue the loop or stop based upon whether the LLM made a tool call"""

    messages = state["messages"]
    last_message = messages[-1]
    # If the LLM makes a tool call, then perform an action
    if last_message.tool_calls:
        return "Action"
    # Otherwise, we stop (reply to the user)
    return END


# Build workflow
agent_builder = StateGraph(MessagesState)

# Add nodes
agent_builder.add_node("llm_call", llm_call)
agent_builder.add_node("environment", tool_node)

# Add edges to connect nodes
agent_builder.add_edge(START, "llm_call")
agent_builder.add_conditional_edges(
    "llm_call",
    should_continue,
    {
        # Name returned by should_continue : Name of next node to visit
        "Action": "environment",
        END: END,
    },
)
agent_builder.add_edge("environment", "llm_call")

memory = MemorySaver()

# Compile the agent
agent = agent_builder.compile(checkpointer=memory)

In [62]:
print("Chat started! Type 'exit' to end.\n")

# Set a default thread ID (can be dynamic if needed)
config = {"configurable": {"thread_id": "1"}}

while True:
    user_input = input("You: ")

    if user_input.lower() in ["exit", "quit"]:
        print("Chat ended.")
        break

    input_message = {"role": "user", "content": user_input}
    
    try:
        for chunk in agent.stream({"messages": [input_message]}, config, stream_mode="values"):
            chunk["messages"][-1].pretty_print()
    except Exception as e:
        print(f"An error occurred: {e}")
        

Chat started! Type 'exit' to end.



KeyboardInterrupt: Interrupted by user

In [63]:
list(agent.stream({"messages": [input_message]}, config, stream_mode="values"))

documents [Document(id='428b942b-e8f0-4fe0-b323-cfeee4fd0962', metadata={'author': '', 'creationdate': '', 'creator': 'LaTeX with hyperref', 'keywords': '', 'page': 49, 'page_label': '50', 'producer': 'pdfTeX-1.40.25', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'source': 'c:\\Users\\DELL\\Desktop\\CloneFlow\\backend\\uploads\\2501.10893v1.pdf', 'subject': '', 'title': '', 'total_pages': 53, 'trapped': '/False'}, page_content='Learn-by-interact: A Data-Centric Framework for Self-Adaptive Agents in Realistic Environments\nInstruction\nWhat are items ordered in the latest cancelled order?\nObservation 0\nAction 1\nclick [4918] // click the button "My Account"\nObservation 1\nTable 42|WebArena example (filtered)\n50'), Document(id='5a713262-20d0-41e3-baa8-4bf8f3154668', metadata={'author': '', 'creationdate': '', 'creator': 'LaTeX with hyperref', 'keywords': '', 'page': 52, 'page_label': '53', 'producer': 'pdfTeX-1.40.25', '

[{'messages': [HumanMessage(content='where do i live', additional_kwargs={}, response_metadata={}, id='09eb5e0c-4516-410b-9dc8-8a4f6551c9c9')]},
 {'messages': [HumanMessage(content='where do i live', additional_kwargs={}, response_metadata={}, id='09eb5e0c-4516-410b-9dc8-8a4f6551c9c9'),
   AIMessage(content='', additional_kwargs={'function_call': {'name': 'RAG_tool', 'arguments': '{"query": "What is my address?"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []}, id='run-cada1267-4cf4-4fff-b29e-5eb00a351558-0', tool_calls=[{'name': 'RAG_tool', 'args': {'query': 'What is my address?'}, 'id': '2e51c9a8-9e65-4c3e-bc86-0bb6665fa1f7', 'type': 'tool_call'}], usage_metadata={'input_tokens': 208, 'output_tokens': 9, 'total_tokens': 217, 'input_token_details': {'cache_read': 0}})]},
 {'messages': [HumanMessage(content='where do i live', additional_kwargs={}, response_metadata={}, 

In [64]:
def generateresponse(promt):
    print("Chat started! Type 'exit' to end.\n")

# Set a default thread ID (can be dynamic if needed)
    config = {"configurable": {"thread_id": "1"}}


    user_input = promt

  

    input_message = {"role": "user", "content": user_input}
        
    try:
            for chunk in agent.stream({"messages": [input_message]}, config, stream_mode="values"):
                chatHistory=list(agent.stream({"messages": [input_message]}, config, stream_mode="values"))
            return chatHistory[-1]["messages"][-1].content            
    except Exception as e:
            print(f"An error occurred: {e}")
            

In [70]:
asd=generateresponse("what is the last email I received?")

Chat started! Type 'exit' to end.



In [72]:
asd

'The last email you received was from GeeksforGeeks, with the subject "Ending Soon... 💪".'

Chat started! Type 'exit' to end.

================================ Human Message =================================

what whas my previous question
================================== Ai Message ==================================

Your previous question was "what is 2+2".
